<a href="https://colab.research.google.com/github/Bhairavi-shah/AbstractiveUserReviewConsolidation/blob/master/Module2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%time
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
"""
df = pd.read_json(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/AMAZON_FASHION_5.json", lines = True)
print(df.head())

df.groupby('asin').mean()

df2 = (df.loc[df['asin'] == 'B0014F7B98'])
print(df2.head())

df2.to_json(F"/content/gdrive/My Drive/PROJECT | S7-S8/Data/B0014F7B98.json", orient ='index')

df3 = pd.read_csv(F"/content/gdrive/My Drive/PROJECT | S7-S8/Data/twitter_sentiment_train.csv", engine='python')
print(df3.head())
"""

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

import re

df = pd.read_csv(F"/content/gdrive/My Drive/PROJECT | S7-S8/Data/training.csv", engine='python', 
                  names=["target", "ids", "date", "flag", "user", "text"])
"""
target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
ids: The id of the tweet ( 2087)
date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
flag: The query (lyx). If there is no query, then this value is NO_QUERY.
user: the user that tweeted (robotickilldozr)
text: the text of the tweet (Lyx is cool)
"""

pd.set_option('display.max_colwidth',1000)
print(df.head())
print(df.columns)
nltk.download('stopwords')
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

# TEXT CLEANING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

df['PreprocessedText'] = df.text.apply(lambda x: preprocess(x))
print(df)
print(df.columns)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
   target  ...                                                                                                                 text
0       0  ...  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
1       0  ...      is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
2       0  ...                            @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
3       0  ...                                                                      my whole body feels itchy and like its on fire 
4       0  ...      @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 

[5 rows x 6 columns]
Index(['target', 'ids', 'date', 'flag', 'user', 'text']

In [24]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


# Encoding string valued columns
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in df.columns:
  if df[column_name].dtype == object:
    df[column_name] = le.fit_transform(df[column_name])
  else:
    pass


# Split target column
y = df[['target']]
df_traintest = df.drop(columns=['target'])

# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(df_traintest, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1280000, 6) (1280000, 1)
(320000, 6) (320000, 1)


In [36]:
#fit a model
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
predictions = text_classifier.predict(X_test)

In [43]:
# Score
print("Score:", text_classifier.score(X_test, y_test))

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

Score: 0.725825
[[102012  57916]
 [ 29820 130252]]
              precision    recall  f1-score   support

           0       0.77      0.64      0.70    159928
           4       0.69      0.81      0.75    160072

    accuracy                           0.73    320000
   macro avg       0.73      0.73      0.72    320000
weighted avg       0.73      0.73      0.72    320000

0.725825
